# 🐀 YOLOv5 Training for Rat and Squirrel Detection
This Colab notebook trains YOLOv5s on a Pascal VOC-style dataset (rat and squirrel), evaluates performance (F1 score, mAP), and prepares the data using MAIXhub output.

In [ ]:
# Clone YOLOv5 and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Total 17483 (delta 0), reused 0 (delta 0), pack-reused 17483 (from 1)
Receiving objects: 100% (17483/17483), 16.58 MiB | 12.10 MiB/s, done.
Resolving deltas: 100% (11991/11991), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.9 MB/s eta 0:00:00
   ━

In [ ]:
# Upload and extract your MAIXhub dataset
from google.colab import files
uploaded = files.upload()

Saving mixed_V2.zip to mixed_V2 (1).zip


In [ ]:
# Unzip the uploaded dataset
import zipfile
with zipfile.ZipFile("mixed_V2.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
import os

for root, dirs, files in os.walk("/content/"):
    print(f"📂 {root}")
    for f in files:
        print(f"   - {f}")


📂 /content/
   - train.txt
   - val.txt
📂 /content/.config
   - gce
   - hidden_gcloud_config_universe_descriptor_data_cache_configs.db
   - config_sentinel
   - .last_opt_in_prompt.yaml
   - .last_update_check.json
   - active_config
   - default_configs.db
   - .last_survey_prompt.yaml
📂 /content/.config/logs
📂 /content/.config/logs/2025.05.28
   - 19.28.05.339588.log
   - 19.28.04.170553.log
   - 19.28.13.198628.log
   - 19.27.36.109756.log
   - 19.28.13.838540.log
   - 19.27.56.530404.log
📂 /content/.config/configurations
   - config_default
📂 /content/yolov5
   - benchmarks.py
   - .dockerignore
   - mixed_V2.zip
   - README.zh-CN.md
   - CITATION.cff
   - pyproject.toml
   - detect.py
   - LICENSE
   - requirements.txt
   - hubconf.py
   - train.py
   - tutorial.ipynb
   - .gitattributes
   - val.py
   - .gitignore
   - mixed_V2 (1).zip
   - export.py
   - CONTRIBUTING.md
   - README.md
📂 /content/yolov5/.git
   - HEAD
   - index
   - config
   - description
   - packed-refs
📂 /c

/content/annotations

In [ ]:
import os
import xml.etree.ElementTree as ET
from glob import glob
from tqdm import tqdm

ANNOT_DIR = '/content/annotations'
LABELS_DIR = '/content/mixed_V2/labels'
os.makedirs(LABELS_DIR, exist_ok=True)

CLASSES = ['rat', 'squirrel']

def convert(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    w = int(root.find('size/width').text)
    h = int(root.find('size/height').text)

    result = []
    for obj in root.findall('object'):
        cls = obj.find('name').text
        if cls not in CLASSES:
            continue
        cls_id = CLASSES.index(cls)
        xmlbox = obj.find('bndbox')
        b = [int(xmlbox.find(x).text) for x in ('xmin', 'ymin', 'xmax', 'ymax')]
        x_center = (b[0] + b[2]) / 2.0 / w
        y_center = (b[1] + b[3]) / 2.0 / h
        bw = (b[2] - b[0]) / w
        bh = (b[3] - b[1]) / h
        result.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")
    return result

xml_files = glob(f"{ANNOT_DIR}/*.xml")
print(f"Found {len(xml_files)} XML files.")

for xml_file in tqdm(xml_files):
    label_data = convert(xml_file)
    image_id = os.path.splitext(os.path.basename(xml_file))[0]
    with open(os.path.join(LABELS_DIR, f"{image_id}.txt"), "w") as f:
        f.write("\n".join(label_data))


Found 1118 XML files.


100%|██████████| 1118/1118 [00:00<00:00, 8193.02it/s]


In [ ]:
!find /content -name "train.txt"
!find /content -name "val.txt"


/content/train.txt
/content/val.txt


In [ ]:
from os.path import basename, splitext
from shutil import move
import os

def move_files(txt_path, subset):
    image_src_dir = '/content/mixed_V2/images'
    label_src_dir = '/content/mixed_V2/labels'

    image_dst = f"/content/mixed_V2/images/{subset}"
    label_dst = f"/content/mixed_V2/labels/{subset}"
    os.makedirs(image_dst, exist_ok=True)
    os.makedirs(label_dst, exist_ok=True)

    with open(txt_path, 'r') as f:
        for line in f:
            filename = basename(line.strip())  # e.g., 'image1.jpg'
            base = splitext(filename)[0]       # e.g., 'image1'

            img_path = os.path.join(image_src_dir, filename)
            label_path = os.path.join(label_src_dir, f"{base}.txt")

            try:
                move(img_path, os.path.join(image_dst, filename))
                move(label_path, os.path.join(label_dst, f"{base}.txt"))
            except FileNotFoundError as e:
                print(f"❌ File not found: {e}")


In [ ]:
move_files('/content/train.txt', 'train')
move_files('/content/val.txt', 'val')


❌ File not found: [Errno 2] No such file or directory: '/content/mixed_V2/images/240_F_1047182378_HWKxmd5udZlzwn1Tbrf7U92upGu5qkgD_jpg.rf.bbf78c1576b262315ae42fc7b460ea5a_1.jpg'
❌ File not found: [Errno 2] No such file or directory: '/content/mixed_V2/images/240_F_1047182319_KtCGAy6Px1fJrHI69Xnm13YYVG5BCOlc_jpg.rf.226f8c7a9b57dc3125f5584d71ff78fc_2.jpg'
❌ File not found: [Errno 2] No such file or directory: '/content/mixed_V2/images/20250124_180426_jpg.rf.08e33db718080b92fd04cecc325b75ca_3.jpg'
❌ File not found: [Errno 2] No such file or directory: '/content/mixed_V2/images/240_F_1047182410_WsYDkGYzcm0X6l6VuvrBn8ORg3MW36Dx_jpg.rf.f98712014a7dea07e450b48e56e4d55b_4.jpg'
❌ File not found: [Errno 2] No such file or directory: '/content/mixed_V2/images/240_F_105567416_hfSMoq4Ljti8b7pII621P2zwrvMu6YMe_jpg.rf.322a0cff4c150a66a7438c7e0c59aab0_5.jpg'
❌ File not found: [Errno 2] No such file or directory: '/content/mixed_V2/images/240_F_105567416_hfSMoq4Ljti8b7pII621P2zwrvMu6YMe_jpg.rf.a678642a

In [ ]:
!ls /content/mixed_V2/images/train | head
!ls /content/mixed_V2/labels/train | head


In [ ]:
# Write dataset YAML file
with open('/content/mixed_v2.yaml', 'w') as f:
    f.write('''train: /content/mixed_V2/images/train
val: /content/mixed_V2/images/val

nc: 2
names: ["rat", "squirrel"]
''')

In [ ]:
# Train the YOLOv5s model
!python train.py --img 640 --batch 16 --epochs 100 --data /content/mixed_v2.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --name mixedv2_yolov5s

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-05-30 09:42:18.374314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748598138.396121    5851 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748598138.402577    5851 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
# Evaluate the model
!python val.py --weights runs/train/mixedv2_yolov5s/weights/best.pt --data /content/mixed_v2.yaml --img 640